In [1]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go

In [2]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [3]:
# ดึง keyspace ทั้งหมด
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces;")

# แสดงผลลัพธ์
print("Available Keyspaces:")
for row in rows:
    print("-", row.keyspace_name)

Available Keyspaces:
- system_auth
- system_schema
- system_distributed
- system
- system_traces
- stock_data


In [4]:
import settrade_v2
from settrade_v2 import Investor

investor = Investor(
                app_id="ZGCsmM0t5c9VMLRJ",                                 
                app_secret="T5eQ70WLmOWDpVvV7jicJtspuBR2A361nzVGmR8xTF4=", 
                broker_id="SANDBOX",
                app_code="SANDBOX",
                is_auto_queue = False)

deri = investor.Derivatives(account_no="Hilman-D")            

account_info = deri.get_account_info()

account_info    

{'creditLine': 2000000000.0,
 'excessEquity': 2000000000.0,
 'cashBalance': 2000000000.0,
 'equity': 2000000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 2000000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [5]:
#Get Symbol
mkt_data = investor.MarketData()
res = mkt_data.get_quote_symbol("TRUBB")
print(res)

{'instrumentType': 'STOCK', 'symbol': 'TRUBB', 'high': None, 'low': None, 'last': None, 'average': None, 'change': None, 'percentChange': None, 'totalVolume': 0, 'totalBuyVolume': 0, 'totalSellVolume': 0, 'totalNoSideVolume': 0, 'status': '', 'marketStatus': 'Close', 'securityType': 'CS', 'eps': -0.26896, 'pe': 0.0, 'pbv': 0.17, 'percentYield': 0.0, 'maturityDate': None, 'exercisePrice': None, 'underlying': None, 'underlyingPrice': None, 'intrinsicValue': None, 'theoretical': None, 'moneyness': None, 'lastTradingDate': None, 'toLastTrade': None, 'exerciseRatio': None, 'impliedVolatility': None, 'exchange': None, 'aumSize': None, 'inav': None}


In [6]:
#Get Candlestick
market = investor.MarketData()
res = market.get_candlestick(
symbol="TRUBB",
interval="1d",
limit=1,
normalized=True,)

In [7]:
# 👀 ตรวจสอบก่อนว่า res เป็น dict และมี key 'data' หรือไม่
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,0,1746637200,0.61,0.62,0.6,0.62,203921,124374.8


In [8]:
df.columns

Index(['lastSequence', 'time', 'open', 'high', 'low', 'close', 'volume',
       'value'],
      dtype='object')

In [9]:
# แปลงวันที่ให้เป็น datetime object
df["time"] = pd.to_datetime(df["time"])
df.set_index("time", inplace=True)

# วาดกราฟ
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=df.index,
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    name="Candlestick"
))

fig.add_trace(go.Bar(
    x=df.index,
    y=df["volume"],
    name="Volume",
    yaxis='y2',
    marker_color='rgba(150,150,150,0.3)'
))

fig.update_layout(
    title="กราฟแท่งเทียนหุ้น (จาก Settrade API)",
    xaxis_rangeslider_visible=False,
    yaxis=dict(title="ราคา"),
    yaxis2=dict(title="Volume", overlaying="y", side="right", showgrid=False),
    height=600,
)

fig.show()

In [15]:
keyspace_name = 'stock_data'

session.execute(f"CREATE KEYSPACE IF NOT EXISTS {keyspace_name} WITH REPLICATION = {{ 'class': 'SimpleStrategy', 'replication_factor': 1 }};")
session.set_keyspace(keyspace_name)



In [16]:
# ดึง keyspace ทั้งหมด
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces;")

# แสดงผลลัพธ์
print("Available Keyspaces:")
for row in rows:
    print("-", row.keyspace_name)

Available Keyspaces:
- system_auth
- system_schema
- system_distributed
- system
- system_traces
- stock_data


In [17]:
insert_query = """
INSERT INTO stock_data.candlestick_data (
    symbol, aumSize, average, change, eps, exchange,
    exercisePrice, exerciseRatio, high, impliedVolatility,
    inav, instrumentType, intrinsicValue, last, lastTradingDate,
    low, marketStatus, maturityDate, moneyness, pbv,
    pe, percentChange, percentYield, securityType, theoretical,
    toLastTrade, totalVolume, underlying, underlyingPrice
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
"""

In [18]:
#Get Symbol
mkt_data = investor.MarketData()
res = mkt_data.get_quote_symbol("TRUBB")
print(res)

{'instrumentType': 'STOCK', 'symbol': 'TRUBB', 'high': None, 'low': None, 'last': None, 'average': None, 'change': None, 'percentChange': None, 'totalVolume': 0, 'totalBuyVolume': 0, 'totalSellVolume': 0, 'totalNoSideVolume': 0, 'status': '', 'marketStatus': 'Close', 'securityType': 'CS', 'eps': -0.26896, 'pe': 0.0, 'pbv': 0.17, 'percentYield': 0.0, 'maturityDate': None, 'exercisePrice': None, 'underlying': None, 'underlyingPrice': None, 'intrinsicValue': None, 'theoretical': None, 'moneyness': None, 'lastTradingDate': None, 'toLastTrade': None, 'exerciseRatio': None, 'impliedVolatility': None, 'exchange': None, 'aumSize': None, 'inav': None}


In [19]:
from datetime import datetime

# ฟังก์ชันแปลงค่าเป็นวันที่
def to_date(val):
    try:
        return datetime.strptime(val, "%Y-%m-%d").date() if val else None
    except Exception as e:
        print(f"Error parsing date: {e}")
        return None
try:
    session.execute(insert_query, (
        res.get("symbol"),
        res.get("aumSize"),
        res.get("average"),
        res.get("change"),
        res.get("eps"),
        res.get("exchange"),
        res.get("exercisePrice"),
        res.get("exerciseRatio"),
        res.get("high"),
        res.get("impliedVolatility"),
        res.get("inav"),
        res.get("instrumentType"),
        res.get("intrinsicValue"),
        res.get("last"),
        to_date(res.get("lastTradingDate")),  # แปลงวันที่
        res.get("low"),
        res.get("marketStatus"),
        to_date(res.get("maturityDate")),  # แปลงวันที่
        res.get("moneyness"),
        res.get("pbv"),
        res.get("pe"),
        res.get("percentChange"),
        res.get("percentYield"),
        res.get("securityType"),
        res.get("theoretical"),
        res.get("toLastTrade"),
        res.get("totalVolume"),
        res.get("underlying"),
        res.get("underlyingPrice")
    ))
    print("✅ Inserted TRUBB data successfully.")
except Exception as e:
    print("❌ Insert failed:", e)

❌ Insert failed: not all arguments converted during string formatting
